<a href="https://colab.research.google.com/github/whale1510/movie_genre_multilabel_classification/blob/main/movie_synopsis_webcrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ 분류기에 입력할 raw 데이터 생성을 위한 영화 데이터 웹 크롤러
- [수행기능] : 영화 이름 데이터를 바탕으로 영화 시놉시스를 크롤링하여 raw_data 셋을 생성
- [데이터] : 레포지토리의 '영화정보 리스트' 파일 / KOBIS 사이트에서 다운로드.

- [데이터 출처] : KOBIS

# 데이터 불러오기

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import random

### 아래 dataset은 kobis에서 가져옴 / https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do

In [ ]:
# 엑셀파일이 구버전이라, 한 sheet에 약 65,000 개밖에 저장을 못하여 98,000개에 대한 데이터를 저장하기 위해 sheet가 총 두 개 있었다.
df1 = pd.read_excel(r'C:\Users\Cheol\SKKU\AML_Project\AML_Data\영화정보 리스트.xlsx',
                   sheet_name = '영화정보 리스트')
df2= pd.read_excel(r'C:\Users\Cheol\SKKU\AML_Project\AML_Data\영화정보 리스트.xlsx',
                   sheet_name = '영화정보 리스트_2', header= None
                   )

In [ ]:
# 두 번째 sheet는 1행부터 바로 데이터가 삽입되어 있었기에 column 이름을 지정해 주었다.
df2.columns = ['영화명', '영화명(영문)', '제작연도', '제작국가', '유형', '장르', '제작상태', '감독', '제작사']

In [ ]:
# 아래로 병합
df = pd.concat([df1, df2]) ; df

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,청정구역,NaN,2021.0,한국,단편,NaN,기타,심재훈,NaN
1,엄마 극혐,My Annoying Mother,2022.0,한국,단편,기타,기타,NaN,NaN
2,러브 액츄얼리,Love Actually,2003.0,"미국,영국",장편,"멜로/로맨스,드라마,코미디",개봉,리차드 커티스,NaN
3,올빼미 가족,HUNTED OWL FAMILY,2023.0,한국,단편,NaN,기타,노진,NaN
4,성숙씨의 테트리스,Home Today,2022.0,한국,단편,드라마,기타,목규리,NaN
...,...,...,...,...,...,...,...,...,...
32962,제2회 서울환경영화제[2005.9.8-9.14],2nd Green Film Festival in Seoul,2005.0,NaN,기타,NaN,기타,NaN,NaN
32963,레슨/달이지고 비가 옵니다/저푸른 초원(단편),NaN,2002.0,한국,단편,NaN,기타,NaN,NaN
32964,엔조이/ 특집! 노래자랑(단편),NaN,2001.0,한국,단편,NaN,기타,NaN,NaN
32965,칙칙이의 내일은 참피온,Tomorrow's Champion,1991.0,한국,장편,코미디,개봉,NaN,NaN


In [ ]:
# 인덱스가 0-65000, 0-33000 이렇게 두 개 였기 때문에, 초기화 시켰다.
df.index = [i for i in range(0,98498)]
df['제작연도'] = df['제작연도'].astype('datetime64[ns]')
df['제작연도'] = df['제작연도'].astype('int64')
df.isnull().sum()

In [ ]:
#결측치 제거
df.dropna(subset=['장르'], inplace=True) ; df.shape

(86509, 9)

In [ ]:
# 영화명과 장르 column만 사용할 것이기 때문에 그 둘만 확인한다.
df.isnull().sum()

영화명            0
영화명(영문)    16904
제작연도           0
제작국가         358
유형            69
장르             0
제작상태         385
감독         22656
제작사        71407
dtype: int64

# 데이터 전처리 / 필터링

> 1. 성인물 제거

> 2. 2010~ 이후의 영화만

> 3. 장편 영화만

> 4. 장르가 '기타'인 것들은 제외시킨다.

In [ ]:
# 조건 1 : 성인물 제거
df = df[~df['장르'].str.contains('에로')] ; df.shape

(78769, 9)

In [ ]:
# 조건 2 : 2010년 이후 영화만
df = df[(df['제작연도'] >= 2010) & (df['제작연도'] <= 2023)] ; df

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
1,엄마 극혐,My Annoying Mother,2022,한국,단편,기타,기타,NaN,NaN
4,성숙씨의 테트리스,Home Today,2022,한국,단편,드라마,기타,목규리,NaN
16,밀수,Smugglers,2023,한국,장편,액션,개봉,류승완,(주)외유내강
17,욕망의 덫,suki,2023,필리핀,장편,드라마,기타,NaN,비바 필름
18,금지된 사랑,PARALUMAN,2021,필리핀,장편,드라마,기타,얌 라라나스,비바 필름
...,...,...,...,...,...,...,...,...,...
97828,500일의 썸머+들어는 봤니? 모건 부부+의형제,Package Screening,2010,기타,장편,기타,기타,NaN,NaN
97830,500일의 썸머+들어는봤니? 모건 부부+하모니,Package Screening,2010,기타,장편,기타,기타,NaN,NaN
97831,파라노말 액티비티+주유소 습격사건2+아빠가 여자를 좋아해,Package Screening,2010,기타,장편,기타,기타,NaN,NaN
97832,500일의 썸머+용서는 없다+주유소 습격사건2,Package Screening,2010,기타,장편,기타,기타,NaN,NaN


In [ ]:
# 조건 3 : 장편 영화만 선택
df = df[df['유형'].str.contains('장편')] ; df

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
16,밀수,Smugglers,2023,한국,장편,액션,개봉,류승완,(주)외유내강
17,욕망의 덫,suki,2023,필리핀,장편,드라마,기타,NaN,비바 필름
18,금지된 사랑,PARALUMAN,2021,필리핀,장편,드라마,기타,얌 라라나스,비바 필름
19,천박사 퇴마 연구소: 설경의 비밀,Dr. Cheon And The Lost Talisman,2022,한국,장편,"판타지,액션",개봉,김성식,(주)외유내강
20,싱글 인 서울,Single in Seoul,2023,한국,장편,"멜로/로맨스,코미디",개봉예정,박범수,"(주)디씨지플러스,(주)명필름,(주)인사이트필름"
...,...,...,...,...,...,...,...,...,...
97828,500일의 썸머+들어는 봤니? 모건 부부+의형제,Package Screening,2010,기타,장편,기타,기타,NaN,NaN
97830,500일의 썸머+들어는봤니? 모건 부부+하모니,Package Screening,2010,기타,장편,기타,기타,NaN,NaN
97831,파라노말 액티비티+주유소 습격사건2+아빠가 여자를 좋아해,Package Screening,2010,기타,장편,기타,기타,NaN,NaN
97832,500일의 썸머+용서는 없다+주유소 습격사건2,Package Screening,2010,기타,장편,기타,기타,NaN,NaN


In [ ]:
df = df[['영화명', '장르']] ; df

,영화명,장르
16,밀수,액션
17,욕망의 덫,드라마
18,금지된 사랑,드라마
19,천박사 퇴마 연구소: 설경의 비밀,"판타지,액션"
20,싱글 인 서울,"멜로/로맨스,코미디"
...,...,...
97828,500일의 썸머+들어는 봤니? 모건 부부+의형제,기타
97830,500일의 썸머+들어는봤니? 모건 부부+하모니,기타
97831,파라노말 액티비티+주유소 습격사건2+아빠가 여자를 좋아해,기타
97832,500일의 썸머+용서는 없다+주유소 습격사건2,기타


In [ ]:
# 장르가 '기타'인 행 삭제
df = df[df['장르'] != '기타'] ;df

,영화명,장르
16,밀수,액션
17,욕망의 덫,드라마
18,금지된 사랑,드라마
19,천박사 퇴마 연구소: 설경의 비밀,"판타지,액션"
20,싱글 인 서울,"멜로/로맨스,코미디"
...,...,...
97573,한-중앙아 영상사업-경계없이+한국을 바라보는 또 다른 시선,다큐멘터리
97575,한-중앙아 영상사업-경계없이,다큐멘터리
97659,소리를 찾아서+강연: 독립다큐멘터리 감독을 위한 사운드 특강,다큐멘터리
97663,투실라고+나는 어떻게 전쟁을 기록했나,다큐멘터리


In [ ]:
# 인덱스 초기화
df.index = [i for i in range(len(df['장르']))] ; df

,영화명,장르
0,밀수,액션
1,욕망의 덫,드라마
2,금지된 사랑,드라마
3,천박사 퇴마 연구소: 설경의 비밀,"판타지,액션"
4,싱글 인 서울,"멜로/로맨스,코미디"
...,...,...
23661,한-중앙아 영상사업-경계없이+한국을 바라보는 또 다른 시선,다큐멘터리
23662,한-중앙아 영상사업-경계없이,다큐멘터리
23663,소리를 찾아서+강연: 독립다큐멘터리 감독을 위한 사운드 특강,다큐멘터리
23664,투실라고+나는 어떻게 전쟁을 기록했나,다큐멘터리


In [ ]:
df.to_csv('영화 11966개 제목 장르_2010년이전.csv')

# 크롤링

In [ ]:
df = pd.read_csv(r'C:\Users\Cheol\SKKU\AML_Project\영화 11966개 제목 장르_2010년이전.csv')

In [ ]:
# 네이버에서 영화 시놉시스를 얻는 방법은
# 포털 사이트에 "영화" + "영화 이름" + "정보" 를 입력하면 된다(url의 query 뒷부분을 보면 알 수 있음)
# 이 점을 이용하여 df에 있는 영화 이름들을 {movie_title}에 하나씩 넣는 방식으로 crwaling를 진행했다.

def get_movie_synopsis(movie_title):
    search_url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_sly.hst&fbm=0&acr=2&ie=utf8&query=%EC%98%81%ED%99%94+{movie_title}+%EC%A0%95%EB%B3%B4'

    response = requests.get(search_url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    synopsis = soup.find('p', class_ = 'text _content_text')
    synopsis = synopsis.text if synopsis else None

    time.sleep(3.5)     # 컴퓨터가 아닌척 하려고 잠시 텀을 둔다(3.5초가 가장 안전한 time.sleep이었음. 2초~3초로 하면 엑세스 거부당함)

    return synopsis

In [ ]:
# tqdm를 활용하여 수집의 진행 정도를 파악한다.
# 1,000개 당 35분정도 소요되었다.
# 이런 식으로 하여 21,400 개 + 12,000 개(데이터 frequency를 확인한 후, 부족한 장르에 대하여 추가수집을 진행하였다.)
tqdm.pandas()
sample_df = df[0:1000]
sample_df['시놉시스'] = sample_df['영화명'].progress_apply(get_movie_synopsis)
sample_df.to_csv('0~1000.csv')